# Clasificador de Residuos Local - Versión Limpia

Este notebook está optimizado para ejecutarse localmente en Windows sin conflictos.

**Clases de residuos:**
- cardboard (cartón)
- compost (compost)  
- glass (vidrio)
- metal (metal)
- paper (papel)
- plastic (plástico)
- trash (basura general)

## 📋 Instrucciones de uso:
1. Ejecuta las celdas en orden secuencial
2. La celda 5 copia solo imágenes nuevas (sin duplicar)
3. La celda 6 clasifica automáticamente la primera imagen
4. La celda 7 permite seleccionar cualquier imagen por índice
5. La celda 8 permite clasificar por ruta específica


In [ ]:
# Instalar dependencias (ejecutar solo una vez)
%pip install fastai==2.7.12 opencv-python matplotlib pillow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Importar librerías
from fastai.vision.all import *
import cv2
import matplotlib.pyplot as plt
import zipfile
import io
import os
from PIL import Image

print("✅ Librerías importadas correctamente")


✅ Librerías importadas correctamente


In [3]:
# Descargar repositorio si no existe
import subprocess

if not os.path.exists("fastai-waste-classifier"):
    print("📁 Descargando repositorio...")
    subprocess.run(["git", "clone", "https://github.com/rootstrap/fastai-waste-classifier"], 
                   check=True, capture_output=True)
    print("✅ Repositorio descargado")
else:
    print("✅ Repositorio ya existe")


✅ Repositorio ya existe


In [4]:
# Cargar modelo con solución para Windows
import torch
import pickle
import sys
from pathlib import Path, WindowsPath

def load_model_windows_safe(model_path):
    """Cargar modelo de forma segura en Windows"""
    try:
        # Método 1: Usar torch.load con pickle personalizado
        print("🔄 Intentando carga con torch.load...")
        
        class WindowsUnpickler(pickle.Unpickler):
            def find_class(self, module, name):
                if module == 'pathlib' and name == 'PosixPath':
                    return WindowsPath
                return super().find_class(module, name)
        
        with open(model_path, 'rb') as f:
            unpickler = WindowsUnpickler(f)
            model_data = unpickler.load()
        
        print("✅ Modelo cargado con éxito usando torch.load")
        return model_data
        
    except Exception as e:
        print(f"⚠️ Error con método 1: {e}")
        
        # Método 2: Usar load_learner con monkey patching
        try:
            print("🔄 Intentando con monkey patching...")
            
            import pathlib
            original_posix = pathlib.PosixPath
            
            class FakePosixPath(WindowsPath):
                def __new__(cls, *args, **kwargs):
                    return WindowsPath(*args, **kwargs)
            
            pathlib.PosixPath = FakePosixPath
            from fastai.vision.all import load_learner
            learn_loaded = load_learner(model_path)
            pathlib.PosixPath = original_posix
            
            print("✅ Modelo cargado con éxito usando monkey patching")
            return learn_loaded
            
        except Exception as e2:
            print(f"❌ Error con método 2: {e2}")
            return None

model_path = "fastai-waste-classifier/result-resnet50.pkl"

if os.path.exists(model_path):
    print("🔄 Cargando modelo...")
    learn_loaded = load_model_windows_safe(model_path)
    
    if learn_loaded:
        print("✅ Modelo cargado exitosamente")
        try:
            print(f"📋 Clases disponibles: {learn_loaded.dls.vocab}")
        except:
            print("📋 Modelo cargado (sin información de clases)")
    else:
        print("❌ No se pudo cargar el modelo")
else:
    print(f"❌ Error: No se encontró el modelo en {model_path}")


🔄 Cargando modelo...
🔄 Intentando carga con torch.load...
⚠️ Error con método 1: A load persistent id instruction was encountered,
but no persistent_load function was specified.
🔄 Intentando con monkey patching...
✅ Modelo cargado con éxito usando monkey patching
✅ Modelo cargado exitosamente
📋 Clases disponibles: ['cardboard', 'compost', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [5]:
# Función para clasificar imágenes
def predict_image(image_path):
    """Clasificar una imagen individual"""
    try:
        # Mostrar imagen
        img = Image.open(image_path)
        plt.figure(figsize=(8, 6))
        plt.imshow(img)
        plt.title(f"Imagen: {os.path.basename(image_path)}")
        plt.axis('off')
        plt.show()
        
        # Predecir
        prediction = learn_loaded.predict(image_path)
        num = prediction[1].numpy().tolist()
        confidence = prediction[2].numpy()[num]
        
        print(f"🏷️ Clasificado como: {prediction[0]}")
        print(f"📊 Confianza: {confidence:.2%}")
        
        return prediction[0], num, confidence
        
    except Exception as e:
        print(f"❌ Error clasificando imagen: {e}")
        return None, None, None

print("✅ Función de clasificación definida")


✅ Función de clasificación definida


In [ ]:
# Verificar y organizar imágenes en carpeta - VERSIÓN FINAL
import shutil
import glob
import os

def organize_images_final():
    """Verificar que las imágenes estén en la carpeta organizada"""
    test_photos_dir = "fastai-waste-classifier/test-photos"
    target_folder = "fotos_para_clasificar"
    
    print(f"📁 Verificando organización de imágenes...")
    
    # Verificar que existe la carpeta destino
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
        print(f"📁 Carpeta creada: {target_folder}")
    else:
        print(f"✅ Carpeta existe: {target_folder}")
    
    # Contar imágenes en la carpeta organizada
    images_in_folder = []
    if os.path.exists(target_folder):
        for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.gif']:
            images_in_folder.extend(glob.glob(os.path.join(target_folder, ext)))
    
    print(f"📸 Imágenes en {target_folder}/: {len(images_in_folder)}")
    
    # Si hay pocas imágenes, copiar desde test-photos
    if len(images_in_folder) < 5 and os.path.exists(test_photos_dir):
        print("📋 Copiando imágenes desde test-photos...")
        
        # Obtener imágenes existentes (solo nombres únicos)
        existing_names = set(os.path.basename(img).lower() for img in images_in_folder)
        
        # Obtener imágenes en test-photos
        test_images = []
        for file in os.listdir(test_photos_dir):
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                test_images.append(file)
        
        # Copiar solo las nuevas
        new_copied = 0
        for file in test_images:
            if file.lower() not in existing_names:
                try:
                    source_path = os.path.join(test_photos_dir, file)
                    target_path = os.path.join(target_folder, file)
                    shutil.copy2(source_path, target_path)
                    print(f"✅ Copiada: {file}")
                    new_copied += 1
                except Exception as e:
                    print(f"❌ Error copiando {file}: {e}")
        
        print(f"📊 Nuevas imágenes copiadas: {new_copied}")
    
    # Mostrar imágenes finales
    final_images = []
    seen_names = set()
    
    if os.path.exists(target_folder):
        for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.gif']:
            for img in glob.glob(os.path.join(target_folder, ext)):
                name = os.path.basename(img).lower()
                if name not in seen_names:
                    final_images.append(img)
                    seen_names.add(name)
        
        print(f"\n📸 Imágenes organizadas en {target_folder}/ ({len(final_images)}) - SIN DUPLICADOS:")
        for i, img in enumerate(sorted(final_images), 1):
            print(f"  {i}. {os.path.basename(img)}")
    else:
        print(f"\n❌ No se encontró la carpeta {target_folder}/")

# Ejecutar función
organize_images_final()


📁 Verificando organización de imágenes...
✅ Carpeta existe: fotos_para_clasificar
📸 Imágenes en fotos_para_clasificar/: 0
📋 Copiando imágenes desde test-photos...
✅ Copiada: basura-envase-plastico.jpeg
✅ Copiada: basura-vaso-plastico.jpeg
✅ Copiada: basura1-metal.jpeg
✅ Copiada: botella plastico azul.jpg
✅ Copiada: botella verde.jpeg
✅ Copiada: botella vidrio.jpg
✅ Copiada: botella-plastico.jpeg
✅ Copiada: botella.png
✅ Copiada: brocoli.jpg
✅ Copiada: camisa.jpg
✅ Copiada: carro viejo.jpg
✅ Copiada: cascara-banana.jpeg
✅ Copiada: cascara-banana2.jpeg
✅ Copiada: cascara-banana3.jpeg
✅ Copiada: cascara.png
✅ Copiada: clavos.png
✅ Copiada: envase-plastico.jpeg
✅ Copiada: envase-plastico2.jpeg
✅ Copiada: higienico.png
✅ Copiada: Imagen de WhatsApp 2025-10-21 a las 08.55.14_be760169.jpg
✅ Copiada: lata abierta.jpg
✅ Copiada: lata cocacola.png
✅ Copiada: lata.jpeg
✅ Copiada: lata2.jpeg
✅ Copiada: manzana-mordida.jpeg
✅ Copiada: muñeca.jpg
✅ Copiada: papaya.png
✅ Copiada: papel-sucio.jpeg
✅ C

In [22]:
# Limpiar imágenes duplicadas de la carpeta organizada
import os
import glob

def clean_duplicate_images_in_folder():
    """Eliminar imágenes duplicadas de la carpeta organizada"""
    target_folder = "fotos_para_clasificar"
    
    if not os.path.exists(target_folder):
        print(f"❌ No se encontró la carpeta {target_folder}/")
        return
    
    print(f"🧹 Limpiando imágenes duplicadas en {target_folder}/...")
    
    # Buscar todas las imágenes en la carpeta
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.gif']
    all_images = []
    
    for ext in image_extensions:
        all_images.extend(glob.glob(os.path.join(target_folder, ext)))
        all_images.extend(glob.glob(os.path.join(target_folder, ext.upper())))
    
    # Agrupar por nombre (sin importar mayúsculas/minúsculas)
    image_groups = {}
    for img_path in all_images:
        name = os.path.basename(img_path).lower()
        if name not in image_groups:
            image_groups[name] = []
        image_groups[name].append(img_path)
    
    # Eliminar duplicados (mantener solo el primero)
    removed_count = 0
    for name, paths in image_groups.items():
        if len(paths) > 1:
            print(f"📁 {name}: {len(paths)} copias encontradas")
            # Mantener el primero, eliminar el resto
            for path in paths[1:]:
                try:
                    os.remove(path)
                    print(f"  🗑️ Eliminado: {os.path.basename(path)}")
                    removed_count += 1
                except Exception as e:
                    print(f"  ❌ Error eliminando {path}: {e}")
    
    print(f"✅ Limpieza completada. {removed_count} archivos duplicados eliminados")

# Ejecutar limpieza
clean_duplicate_images_in_folder()

# Mostrar imágenes finales en la carpeta organizada (sin duplicados)
images = []
seen_names = set()
target_folder = "fotos_para_clasificar"

if os.path.exists(target_folder):
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.gif']:
        for img in glob.glob(os.path.join(target_folder, ext)):
            name = os.path.basename(img).lower()
            if name not in seen_names:
                images.append(img)
                seen_names.add(name)
    
    print(f"\n📸 Imágenes finales en {target_folder}/ ({len(images)}) - SIN DUPLICADOS:")
    for i, img in enumerate(sorted(images), 1):
        print(f"  {i}. {os.path.basename(img)}")
else:
    print(f"\n❌ No se encontró la carpeta {target_folder}/")


🧹 Limpiando imágenes duplicadas en fotos_para_clasificar/...
📁 botella plastico azul.jpg: 2 copias encontradas
  🗑️ Eliminado: botella plastico azul.jpg
📁 botella vidrio.jpg: 2 copias encontradas
  🗑️ Eliminado: botella vidrio.jpg
📁 brocoli.jpg: 2 copias encontradas
  🗑️ Eliminado: brocoli.jpg
📁 camisa.jpg: 2 copias encontradas
  🗑️ Eliminado: camisa.jpg
📁 carro viejo.jpg: 2 copias encontradas
  🗑️ Eliminado: carro viejo.jpg
📁 imagen de whatsapp 2025-10-21 a las 08.55.14_be760169.jpg: 2 copias encontradas
  🗑️ Eliminado: Imagen de WhatsApp 2025-10-21 a las 08.55.14_be760169.jpg
📁 lata abierta.jpg: 2 copias encontradas
  🗑️ Eliminado: lata abierta.jpg
📁 muñeca.jpg: 2 copias encontradas
  🗑️ Eliminado: muñeca.jpg
📁 basura-envase-plastico.jpeg: 2 copias encontradas
  🗑️ Eliminado: basura-envase-plastico.jpeg
📁 basura-vaso-plastico.jpeg: 2 copias encontradas
  🗑️ Eliminado: basura-vaso-plastico.jpeg
📁 basura1-metal.jpeg: 2 copias encontradas
  🗑️ Eliminado: basura1-metal.jpeg
📁 botella ver

In [24]:
# 🎯 CLASIFICADOR CON CONTENEDORES VISUALES - INTERFAZ MEJORADA
import glob
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from PIL import Image
import warnings

# Suprimir warnings de matplotlib
warnings.filterwarnings('ignore')

def clasificador_con_contenedores():
    """Clasificador que muestra el contenedor correspondiente con interfaz mejorada"""
    target_folder = "fotos_para_clasificar"
    contenedores_folder = "contenedores"
    
    if not os.path.exists(target_folder):
        print(f"❌ No se encontró la carpeta {target_folder}/")
        return
    
    if not os.path.exists(contenedores_folder):
        print(f"❌ No se encontró la carpeta {contenedores_folder}/")
        return
    
    # Buscar imágenes disponibles
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.gif']
    images = []
    seen_names = set()
    
    for ext in image_extensions:
        for img in glob.glob(os.path.join(target_folder, ext)):
            name = os.path.basename(img).lower()
            if name not in seen_names:
                images.append(img)
                seen_names.add(name)
    
    if not images:
        print(f"❌ No hay imágenes en {target_folder}/")
        return
    
    # Mapeo de clases a contenedores
    contenedor_mapping = {
        'cardboard': 'azul.png',      # Papel y cartón → Azul
        'paper': 'azul.png',          # Papel y cartón → Azul
        'glass': 'verde.png',         # Vidrio → Verde
        'plastic': 'amarillo.png',    # Plástico y metal → Amarillo
        'metal': 'amarillo.png',      # Plástico y metal → Amarillo
        'compost': 'marron.png',      # Orgánico → Marrón
        'trash': 'marron.png'         # Basura general → Marrón
    }
    
    # Crear widgets con mejor estilo
    imagen_dropdown = widgets.Dropdown(
        options=[(os.path.basename(img), img) for img in sorted(images)],
        description='📸 Imagen:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='450px', height='40px')
    )
    
    clasificar_btn = widgets.Button(
        description='🎯 Clasificar',
        button_style='success',
        layout=widgets.Layout(width='150px', height='40px')
    )
    
    # Widget para mostrar información inicial
    info_contenedor = widgets.HTML(
        value="<div style='text-align: center; padding: 15px; background-color: #f0f0f0; border-radius: 8px; font-size: 18px; margin: 10px 0;'><b>Selecciona una imagen para clasificar</b></div>",
        layout=widgets.Layout(width='100%', height='80px')
    )
    
    # Output único para todo el resultado
    resultado_output = widgets.Output(layout=widgets.Layout(width='100%', margin='10px 0'))
    
    def on_clasificar_click(b):
        with resultado_output:
            clear_output(wait=True)
            
            if imagen_dropdown.value:
                selected_image = imagen_dropdown.value
                
                # Clasificar SIN mostrar imagen individual
                try:
                    prediction = learn_loaded.predict(selected_image)
                    num = prediction[1].numpy().tolist()
                    confidence = prediction[2].numpy()[num]
                    clase = prediction[0]
                except Exception as e:
                    print(f"❌ Error clasificando imagen: {e}")
                    clase = None
                    confidence = None
                
                if clase:
                    # Interpretación en español
                    interpretaciones = {
                        'cardboard': '📦 Cartón',
                        'compost': '🌱 Compost/Orgánico',
                        'glass': '🍷 Vidrio',
                        'metal': '🔩 Metal',
                        'paper': '📄 Papel',
                        'plastic': '🥤 Plástico',
                        'trash': '🗑️ Basura General'
                    }
                    
                    interpretacion = interpretaciones.get(clase, clase)
                    
                    # Mostrar contenedor correspondiente
                    contenedor_file = contenedor_mapping.get(clase)
                    if contenedor_file:
                        contenedor_path = os.path.join(contenedores_folder, contenedor_file)
                        if os.path.exists(contenedor_path):
                            # Mostrar imagen del contenedor con mejor diseño
                            try:
                                contenedor_img = Image.open(contenedor_path)
                                
                                # Configurar matplotlib para evitar warnings
                                plt.rcParams['figure.facecolor'] = 'white'
                                plt.rcParams['axes.facecolor'] = 'white'
                                
                                # Imágenes MUCHO más pequeñas
                                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
                                
                                # Imagen de la basura (izquierda) - más grande
                                basura_img = Image.open(selected_image)
                                ax1.imshow(basura_img)
                                ax1.set_title("📸 Residuo Clasificado", fontsize=12, fontweight='bold', pad=10)
                                ax1.axis('off')
                                
                                # Imagen del contenedor (derecha) - más pequeña
                                ax2.imshow(contenedor_img)
                                ax2.set_title(f"🗑️ Contenedor: {interpretacion}", fontsize=12, fontweight='bold', pad=10)
                                ax2.axis('off')
                                
                                plt.tight_layout()
                                plt.show()
                                
                                # Información del contenedor con mejor formato - SOLO DEBAJO DE LAS IMÁGENES
                                colores_contenedores = {
                                    'azul.png': '🔵 AZUL - Papel y Cartón',
                                    'verde.png': '🟢 VERDE - Vidrio',
                                    'amarillo.png': '🟡 AMARILLO - Plástico y Metal',
                                    'marron.png': '🟤 MARRÓN - Orgánico'
                                }
                                
                                info_contenedor_texto = colores_contenedores.get(contenedor_file, contenedor_file)
                                
                                # Mostrar resultado debajo de las imágenes
                                print("\n" + "="*60)
                                print("🎯 RESULTADO DE CLASIFICACIÓN")
                                print("="*60)
                                print(f"🏷️ Clase: {interpretacion}")
                                print(f"📊 Confianza: {confidence:.1%}")
                                print(f"🎨 {info_contenedor_texto}")
                                print("="*60)
                                
                            except Exception as e:
                                print(f"❌ Error mostrando contenedor: {e}")
                        else:
                            print(f"❌ No se encontró el contenedor: {contenedor_file}")
                    else:
                        print(f"❌ No hay contenedor definido para la clase: {clase}")
                else:
                    print("❌ Error en la clasificación")
    
    clasificar_btn.on_click(on_clasificar_click)
    
    # Mostrar interfaz mejorada
    print("🎯 CLASIFICADOR DE RESIDUOS CON CONTENEDORES")
    print("=" * 60)
    print("📋 INSTRUCCIONES:")
    print("   1. Selecciona una imagen del menú desplegable")
    print("   2. Haz clic en 'Clasificar'")
    print("   3. Verás la basura y el contenedor correspondiente")
    print("=" * 60)
    
    # Layout mejorado
    header_widget = widgets.HTML(
        value="<h2 style='text-align: center; color: #1976D2; margin: 10px 0;'>🗑️ Clasificador de Residuos</h2>",
        layout=widgets.Layout(width='100%')
    )
    
    controls_widget = widgets.HBox([imagen_dropdown, clasificar_btn])
    controls_widget.layout.justify_content = 'center'
    
    # Mostrar widgets en orden con separación clara - MENSAJE DEBAJO DEL TÍTULO
    display(header_widget)
    display(info_contenedor)
    display(controls_widget)
    
    # Separador visual
    separator = widgets.HTML(
        value="<hr style='margin: 20px 0; border: 1px solid #ddd;'>",
        layout=widgets.Layout(width='100%')
    )
    display(separator)
    
    # Mostrar output único
    display(resultado_output)

# Ejecutar el clasificador con contenedores
clasificador_con_contenedores()


🎯 CLASIFICADOR DE RESIDUOS CON CONTENEDORES
📋 INSTRUCCIONES:
   1. Selecciona una imagen del menú desplegable
   2. Haz clic en 'Clasificar'
   3. Verás la basura y el contenedor correspondiente


HTML(value="<h2 style='text-align: center; color: #1976D2; margin: 10px 0;'>🗑️ Clasificador de Residuos</h2>",…

HTML(value="<div style='text-align: center; padding: 15px; background-color: #f0f0f0; border-radius: 8px; font…

HTML(value="<hr style='margin: 20px 0; border: 1px solid #ddd;'>", layout=Layout(width='100%'))

Output(layout=Layout(margin='10px 0', width='100%'))